In [82]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [83]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [84]:
from pyspark.sql.functions import *

filepath='file:////home/talentum/shared/Project/Project_codes/section5/15stocks_price.csv'
df = spark.read.option("header", True).csv(filepath)
df = df.withColumn("date_gsheets",to_date(df.date_gsheets,"MM/dd/yyyy HH:mm"))
df = df.withColumnRenamed('BRK.B', 'BRK')
df.show()

+------------+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+------+-----+-----+-----+
|date_gsheets| AAPL|   KO| NFLX|  BRK|  DIS|   IBM|   VZ|  WMT|   GE| TSLA|   MA|  AMZN| MSFT|   UN|    V|
+------------+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+------+-----+-----+-----+
|  2012-01-03|58.75|35.07|10.32|77.68|38.31| 186.3|39.73|60.33|18.36|28.08|36.84|179.03|26.77|34.92|25.75|
|  2012-01-04|59.06|34.85|11.49| 76.8|38.85|185.54|39.21|59.71|18.56|27.71|35.63|177.51| 27.4|34.45|25.29|
|  2012-01-05|59.72|34.69|11.33|76.93| 39.5|184.66|38.94|59.42|18.55|27.12|35.24|177.61|27.68|34.49|25.48|
|  2012-01-06|60.34|34.47|12.33|76.39|39.91|182.54|38.33|   59|18.65|26.91|34.29|182.61|28.11|33.27|25.18|
|  2012-01-09|60.25|34.47|14.03|76.29|39.75|181.59|38.37|59.18|18.86|27.25|34.58|178.56|27.74|33.72|24.98|
|  2012-01-10|60.46|34.67|13.69|77.42|39.63|181.31|38.57|59.04|18.72|27.62|34.88|179.34|27.84|33.91|24.95|
|  2012-01-11|60.36|34.03|13.16|   78

In [85]:
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import functions as F

return_df=df
my_window = Window.partitionBy().orderBy("date_gsheets")
for col_name in return_df.columns:
    if col_name!='date_gsheets':
     return_df = return_df.withColumn(col_name, F.when(F.isnull(col(col_name) - F.lag(col(col_name)).over(my_window)), 0)
                                      .otherwise(col(col_name) - F.lag(col(col_name)).over(my_window))/F.lag(col(col_name)).over(my_window))
for c in return_df.columns:
    if c!='date_gsheets':
        return_df = return_df.withColumn(c, F.round(c, 6))

return_df=return_df.dropna()
return_df=return_df.drop('date_gsheets')
return_df.show()

+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|     AAPL|       KO|     NFLX|      BRK|      DIS|      IBM|       VZ|      WMT|       GE|     TSLA|       MA|     AMZN|     MSFT|       UN|        V|
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| 0.005277|-0.006273| 0.113372|-0.011329| 0.014096|-0.004079|-0.013088|-0.010277| 0.010893|-0.013177|-0.032845| -0.00849| 0.023534|-0.013459|-0.017864|
| 0.011175|-0.004591|-0.013925| 0.001693| 0.016731|-0.004743|-0.006886|-0.004857| -5.39E-4|-0.021292|-0.010946|  5.63E-4| 0.010219| 0.001161| 0.007513|
| 0.010382|-0.006342| 0.088261|-0.007019|  0.01038|-0.011481|-0.015665|-0.007068| 0.005391|-0.007743|-0.026958| 0.028152| 0.015535|-0.035373|-0.011774|
|-0.001492|      0.0| 0.137875|-0.001309|-0.004009|-0.005204| 0.001044| 0.003051|  0.011

In [86]:
data1=return_df.columns
data2=[]
for c in return_df.columns:
    data2.append(return_df.select(mean(col(c))).collect()[0][0]) 
data = list(zip(data1, data2))
return_df_mean=spark.createDataFrame(data,["Stocks","Mean"])
return_df_mean = return_df_mean.withColumn("Mean", round(return_df_mean["Mean"], 6))
return_df_mean.show()


+------+--------+
|Stocks|    Mean|
+------+--------+
|  AAPL| 8.25E-4|
|    KO| 2.16E-4|
|  NFLX|0.002453|
|   BRK| 6.61E-4|
|   DIS|  7.5E-4|
|   IBM| -6.2E-5|
|    VZ| 2.42E-4|
|   WMT| 3.84E-4|
|    GE|  3.4E-5|
|  TSLA|0.002057|
|    MA| 0.00102|
|  AMZN|0.001416|
|  MSFT| 8.68E-4|
|    UN| 3.88E-4|
|     V|0.001068|
+------+--------+



In [87]:
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.sql.functions import sqrt
from pyspark.mllib.linalg import DenseMatrix

def getPortReturn(weights):
    """
    Returns the Annualised Expected Return of a portfolio.
    Annualises the return using the 'crude' method.
    """
    return_df_mean_list = return_df_mean.select("Mean").rdd.flatMap(lambda x: x).collect()
    weight_matrix= DenseMatrix(1, num_stocks, weights)
    weight_row_mat = RowMatrix(sc.parallelize(weight_matrix.toArray().tolist()))
    return_df_matrix=DenseMatrix(return_df_mean.count(),1, return_df_mean_list)
    exp_ret_portfolio=weight_row_mat.multiply(return_df_matrix).rows.collect()
    exp_ret_portfolio_annual = exp_ret_portfolio[0][0] * 250
    
    
    return exp_ret_portfolio_annual

In [88]:
# Create a vector of equal weights as the initial (init) guess
num_stocks = len(return_df.columns)
init_weights = [1 / num_stocks] * num_stocks
from pyspark.mllib.linalg import DenseMatrix
weight_matrix= DenseMatrix(1, num_stocks, init_weights)


In [89]:
getPortReturn(init_weights)

0.20533333333333334

In [90]:
target_return = 0.4

In [91]:
bounds = tuple((0, 1) for i in range(num_stocks))

In [100]:
# Setup the other 2 constraints
cons = (
    # Sum of weights must equate to 1
    {'type' : 'eq', 'fun' : lambda w : np.sum(w) - 1},
    
    # Difference between expected return and target must be equal to 0.
    {'type' : 'eq', 'fun' : lambda x : getPortReturn(x) - target_return})

In [101]:
from scipy.optimize import minimize
import numpy as np
results = minimize(fun=getPortReturn,  # being the objective function
                   x0=init_weights,  # being the initial guess
                   # bounds: being the constraint that the weight of any asset i
                   # must be between 0 and 1 inclusive
                   bounds=bounds,
                   constraints=cons)  # being the other 2 constraints (see 'cons' in In[13])

In [94]:
results

     fun: 0.40000000017833076
     jac: array([ 0.20625,  0.054  ,  0.61325,  0.16525,  0.1875 , -0.0155 ,
        0.0605 ,  0.096  ,  0.0085 ,  0.51425,  0.255  ,  0.354  ,
        0.217  ,  0.097  ,  0.267  ])
 message: 'Optimization terminated successfully'
    nfev: 32
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([6.01162975e-02, 5.55111512e-17, 2.56008789e-01, 4.03826543e-02,
       5.10917650e-02, 2.77555756e-17, 0.00000000e+00, 7.05205262e-03,
       0.00000000e+00, 2.08359264e-01, 8.35800772e-02, 1.31229602e-01,
       6.52903601e-02, 7.53336014e-03, 8.93557773e-02])

In [95]:
getPortReturn(weights=results['x'])

0.40000000017833076

In [96]:
rdd=sc.parallelize(results['x'])
optimised_weights=spark.createDataFrame(rdd.map(lambda x: float(x)), FloatType())
optimised_weights=optimised_weights.withColumnRenamed("value","Optimized_Weights")
optimised_weights.show()


#optimised_weights = spark.createDataFrame(results['x'])

+-----------------+
|Optimized_Weights|
+-----------------+
|        0.0601163|
|     5.551115E-17|
|        0.2560088|
|      0.040382653|
|      0.051091764|
|    2.7755576E-17|
|              0.0|
|     0.0070520528|
|              0.0|
|       0.20835927|
|       0.08358008|
|       0.13122961|
|       0.06529036|
|     0.0075333603|
|      0.089355774|
+-----------------+



In [97]:
# Create a dataframe of all the portfolio risks together with protfolio columns

stocks_df = spark.createDataFrame([(l,) for l in return_df.columns], ["Stocks"])

# Add sequential index to both DataFrames
optimised_weights= optimised_weights.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
stocks_df = stocks_df.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

# Join DataFrames on the index and drop the index column
optimised_weights_df = stocks_df.join(optimised_weights, "row_idx").drop("row_idx")

optimised_weights_df = optimised_weights_df.withColumn("Optimized_Weights", round(optimised_weights_df["Optimized_Weights"], 6))

# Show the result
optimised_weights_df.show()



+------+-----------------+
|Stocks|Optimized_Weights|
+------+-----------------+
|  AAPL|         0.060116|
|    KO|              0.0|
|  NFLX|         0.256009|
|   BRK|         0.040383|
|   DIS|         0.051092|
|   IBM|              0.0|
|    VZ|              0.0|
|   WMT|         0.007052|
|    GE|              0.0|
|  TSLA|         0.208359|
|    MA|          0.08358|
|  AMZN|          0.13123|
|  MSFT|          0.06529|
|    UN|         0.007533|
|     V|         0.089356|
+------+-----------------+

